In [52]:
import pymysql as mariadb
import pandas as pd
import matplotlib
import random
import re
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Zeitreihe erstellen

Der nachfolgende Code erstellt basierend auf den gescrapten Artikel und Kommentare als auch den Daten vom BAG eine Zeitreihe.

In [53]:
#get all days as a list

from datetime import date, timedelta

sdate = date(2020, 1, 1)   # start date
edate = date(2020, 12, 31)   # end date

delta = edate - sdate       # as timedelta

kalender = []
for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    kalender.append(day)

In [54]:
#establish dataframe from list
df = pd.DataFrame(kalender)
df = df.set_index(df[0]).astype(str)
df = df.rename({0: "tage"}, axis = "columns")
df

,tage
0,
2020-01-01,2020-01-01
2020-01-02,2020-01-02
2020-01-03,2020-01-03
2020-01-04,2020-01-04
2020-01-05,2020-01-05
...,...
2020-12-27,2020-12-27
2020-12-28,2020-12-28
2020-12-29,2020-12-29


In [55]:
#artikel
connection = mariadb.connect(host="localhost", database="masterarbeit",
                            user="adriana", password="Welcome.2017.")

#filtering out the datasources where there are no comments or comments without date
sql = ("SELECT * "
       "FROM article "
       "WHERE newsoutlet NOT Like 'nau' "
       "AND newsoutlet NOT like 'nlz' "
       "AND newsoutlet NOT like 'woz' ")

df_artikel = pd.read_sql_query(sql, connection)

df_artikel_onlywithdate = df_artikel
df_artikel_onlywithdate["date"] = df_artikel_onlywithdate["art_datum"].astype(str)
df_artikel_onlywithdate = df_artikel_onlywithdate.sort_values(by="date")
df_artikel_onlywithdate = df_artikel_onlywithdate.mask(df_artikel_onlywithdate.eq('None')).dropna(subset=["date"])

df_artikel_onlywithdate = df_artikel_onlywithdate.set_index("date")

df_art_zf = df_artikel_onlywithdate.groupby(["date"]).size()
df_art_zf  = df_art_zf.to_frame()

In [56]:
#connecting to commentaires
sql2 = ("SELECT * "
        "FROM comment "
        "WHERE outlet NOT Like 'nau' ")

df_komm = pd.read_sql_query(sql2, connection)

df_komm_onlywithdate = df_komm
df_komm_onlywithdate["date"] = df_komm_onlywithdate["kom_datum"].astype(str)

df_komm_onlywithdate = df_komm_onlywithdate.sort_values(by="date")
df_komm_onlywithdate = df_komm_onlywithdate.set_index("date")

df_komm_zf = df_komm_onlywithdate.groupby(["date"]).size()
df_komm_zf = df_komm_zf.to_frame()

In [57]:
#fallzahlen

sql3 = ("SELECT * "
       "FROM bag_fallzahlen "
       "WHERE geoRegion = 'CH'")

df_fallzahlen = pd.read_sql_query(sql3, connection)

df_fallzahlen = df_fallzahlen.sort_values(by = "datum", ascending = True)
df_fallzahlen["datum"] = df_fallzahlen["datum"].astype(str)
df_fallzahlen
df_fallzahlen = df_fallzahlen.rename(columns = {"entries": "entries_fall"})
df_fall2 = df_fallzahlen[["datum", "entries_fall"]].copy()

In [58]:
#hospzahlen

sql3 = ("SELECT * "
       "FROM bag_hospzahlen "
       "WHERE geoRegion = 'CH'")

df_hospzahlen = pd.read_sql_query(sql3, connection)

df_hospzahlen["hosp_datum"] = pd.to_datetime(df_hospzahlen["hosp_datum"])
df_hospzahlen["hosp_datum"] = df_hospzahlen["hosp_datum"].astype(str)
df_hospzahlen = df_hospzahlen.rename(columns = {"entries": "entries_hosp"})
df_hosp2 = df_hospzahlen[["hosp_datum", "entries_hosp"]].copy()

In [59]:
#todeszahlen

sql4 = ("SELECT * "
       "FROM bag_todzahlen "
       "WHERE geoRegion = 'CH'")

df_todzahlen = pd.read_sql_query(sql4, connection)
df_todzahlen["datum"] = pd.to_datetime(df_todzahlen["datum"], format="%d.%m.%Y")
df_todzahlen["datum"] = df_todzahlen["datum"].astype(str)
df_todzahlen = df_todzahlen.rename(columns = {"entries": "entries_tod"})
df_tod = df_todzahlen[["datum", "entries_tod"]]

In [60]:
df_tod

,datum,entries_tod
0,2020-02-24,0
1,2020-02-25,0
2,2020-02-26,0
3,2020-02-27,0
4,2020-02-28,0
...,...,...
307,2020-12-27,91
308,2020-12-28,79
309,2020-12-29,76
310,2020-12-30,92


In [61]:
final = df.merge(df_art_zf, right_on = "date", left_on = "tage", how ="left")
#final = df_komm_zf

In [62]:
final = final.merge(df_komm_zf, right_on = "date", left_on = "tage", how = "left", suffixes = (None, "_kom"))

In [63]:
final = final.merge(df_fall2, left_on = "tage", right_on="datum", how = "left")

In [64]:
final = final.merge(df_hosp2, left_on = "tage", right_on="hosp_datum", how="left")

In [65]:
final = final.merge(df_tod, left_on = "tage", right_on="datum", how="left")

In [66]:
final

,tage,0,0_kom,datum_x,entries_fall,hosp_datum,entries_hosp,datum_y,entries_tod
0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
361,2020-12-27,14.0,907.0,2020-12-27,2114,2020-12-27,124.0,2020-12-27,91
362,2020-12-28,35.0,884.0,2020-12-28,5032,2020-12-28,219.0,2020-12-28,79
363,2020-12-29,35.0,1066.0,2020-12-29,4462,2020-12-29,156.0,2020-12-29,76
364,2020-12-30,42.0,1206.0,2020-12-30,4210,2020-12-30,165.0,2020-12-30,92


In [67]:
final = final.drop(['datum_x', 'hosp_datum', 'datum_y'], axis=1)
final = final.fillna(0)

final["entries_fall"] = final["entries_fall"].astype(float)
final["entries_tod"] = final["entries_fall"].astype(float)
final["tage"] = pd.to_datetime(final["tage"], format="%Y-%m-%d")
final = final.set_index("tage")
weekly = final.groupby(pd.Grouper(freq='w')).sum()

In [42]:
weekly.to_csv("weekly.csv")